In [1]:
# libraries and settings
import os
import c3d
import glob2
import math
import numpy as np
# from mayavi.mlab import *
import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.animation as animation
from matplotlib import animation, rc
from mpl_toolkits.mplot3d import Axes3D

# linear regression with sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from IPython.display import HTML, Image

%matplotlib qt

rc('animation', html='html5')

In [80]:
# try to read c3d files
folder_name = 'files_motions_589'
cd3data_path = os.path.join('data', 'motion_data', folder_name)
all_files = glob2.glob(cd3data_path+'/**/*[0-9].c3d', recursive=True)

print("Loading files....")
for files in all_files: 
    print(files)

# read c3d file
file_id = 0
reader = c3d.Reader(open(all_files[file_id], 'rb'))

Loading files....
data/motion_data/files_motions_589/pour02.c3d
data/motion_data/files_motions_589/pour04.c3d
data/motion_data/files_motions_589/pour03.c3d
data/motion_data/files_motions_589/pour01.c3d
data/motion_data/files_motions_589/pour05.c3d
data/motion_data/files_motions_589/pour06.c3d


### See information about marker points [here](https://motion-database.humanoids.kit.edu/marker_set/)

In [81]:
# function to read points from file
def read_c3d(c3d_file, objs_id_end, rh_id_start, lh_id_start, verbose=False):
    """
    read a c3d file and output the groups in the scene with the path of the end point
    """
    # read file
    reader = c3d.Reader(open(c3d_file, 'rb'))
    
    # reader.point_used
    point_labels = reader.point_labels
    dis = 8 # distance of the endpoint id
    
    if verbose:
        print("\nRight arm points:")
        print(point_labels[rh_id_start:rh_id_start+10])

        print("\nLeft arm points:")
        print(point_labels[lh_id_start:lh_id_start+10])

        print("\nEnd point markers:")
        print(point_labels[rh_id_start+dis:rh_id_start+dis+2])
        print(point_labels[lh_id_start+dis:lh_id_start+dis+2])
    
    object_points_ids = list(range(objs_id_end))
    left_arm_points_ids = list(range(lh_id_start,lh_id_start+10)) #28
    right_arm_points_ids = list(range(rh_id_start,rh_id_start+10)) #38

    lh_traj_points_id = list(range(lh_id_start+dis,lh_id_start+dis+2))
    rh_traj_points_id = list(range(rh_id_start+dis,rh_id_start+dis+2))

    scene_groups_ids = [object_points_ids, left_arm_points_ids, right_arm_points_ids]
    traj_groups_ids = [lh_traj_points_id, rh_traj_points_id]
    
    # collect points of interest
    scene_groups = []
    for group in scene_groups_ids:
        group_points = []
        for point_id in group:
            point_marker = []
            for i, points, analog in reader.read_frames():
                point_marker.append(points[point_id][:3])
            group_points.append(point_marker)
        scene_groups.append(np.array(group_points))
        
    # for trajectory points that we want to trace
    traj_groups = []
    for group in traj_groups_ids:
        traj_points = []
        for point_id in group:
            point_marker = []
            for i, points, analog in reader.read_frames():
                traj_mrkrs = points[point_id][:3]
                point_marker.append(traj_mrkrs)
            traj_points.append(point_marker)
        traj_groups.append(np.array(traj_points))
    
    # we want to create path from the traj_points
    mid_points = []
    for group in traj_groups:
        point_a = group[0]
        point_b = group[1]
        mid_points.append((point_a + point_b)/2.0)

    
    
    return scene_groups, mid_points

In [82]:
# read c3d file
objs_id_end = 12
lh_id_start = 21
rh_id_start = 31
# dis = 8 # distance of the endpoint id

all_scene_groups = []
# all_mid_points = []
lh_paths = []
rh_paths = []
for files in all_files:
    scene_groups, mid_points = read_c3d(files, 
                                        objs_id_end, 
                                        rh_id_start, 
                                        lh_id_start,
                                        verbose=0)
    lh_paths.append(mid_points[0])
    rh_paths.append(mid_points[1])

important segments:

0 -- [190:333], [333:460]

In [83]:
# generic plot function
def plot_3d(data_list, verbose=False, full_screen=False):
    """
    data is a tuple of x, y, z
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    if full_screen:
        figManager = plt.get_current_fig_manager()
        figManager.window.showMaximized()
    if verbose:
        ax.set_xlim3d([-1700.0, 0.0])
        ax.set_xlabel('X')

        ax.set_ylim3d([-600.0, 800.0])
        ax.set_ylabel('Y')

        ax.set_zlim3d([700.0, 1600.0])
        ax.set_zlabel('Z')
    ax.azim = 160 #200
    ax.elev = 30
    
    
    for data in data_list:
        x = data[:,0]
        y = data[:,1]
        z = data[:,2]
        ax.plot(x, y, z)

def plot_line(data, ln=None):
    if ln==None:
        fig = plt.figure()
        ln = fig.add_subplot(111)
    ln.plot(data)
    return ln

def plot_scatter(x, y, ln=None):
    if ln==None:
        fig = plt.figure()
        ln = fig.add_subplot(111)
    ln.scatter(x, y)
    return ln

In [144]:
def poly_func(t, params):
    y = params[0]
    for i in range(1, len(params)):
        y = y + (params[i]*t**i)
    return y

# make features from t
def make_features(t, order=3, fixed=None):
    """
    fixed should be a list of 0 and 1 signifying which coefficients to set to 0
    """
    if fixed == None:
        T1 = np.ones_like(t)
        T2 = t
    else:
        
    T_array = [T1, T2]
    for i in range(2, order+1):
        T_array.append(T2**i)
    T=np.transpose(np.array(T_array))
    if fixed!=None:
        
    return T

# analytical solution using the normal equation: (X^T*X)^-1*X^T*Y
def solve_poly(var, t, order=3):
    T = make_features(t, order)
    T_transpose = np.transpose(T)
    params = np.matmul(np.linalg.inv(np.matmul(T_transpose,T)), 
                       np.matmul(T_transpose, var))
    new_x = poly_func(t, params)
    
    return new_x

In [163]:
# extract a sample paths and time steps [190-270-330-450]
# we are interested in the trajectories which involve picking and pouring
# placing back can be easily executed
sample_id = 0
paths = rh_paths[sample_id]

start_point = 200
cut_point = 270
end_point = 330
final_point = 460
full_path = paths[start_point:final_point]
segment_1 = paths[start_point:cut_point]
# t_1_o = np.array(range(start_point,cut_point))
t_1 = np.array(range(cut_point-start_point))

segment_2 = paths[cut_point:end_point] # segment 2
# t_2_o = np.array(range(cut_point,end_point))
# t_2 = np.array(range(end_point-cut_point))
t_2 = np.array(range(t_1[-1]+1,t_1[-1]+1+end_point-cut_point))
# t_2 = np.array(list(range(cut_point, end_point)))

segment_3 = paths[end_point:final_point] # returning trajectory
t_3 = np.array(range(final_point-end_point))

# pl = plot_3d([segment_1, segment_2, segment_3], full_screen=True)
# pl = plot_3d([full_path], full_screen=True)
# print(t_1, t_1_o)
# print(t_2, t_2_o)
print(t_1)
print(t_2)
# print(t_3)
# using the real time steps gave the same result as using a relative time step for each trajectory segment

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69]
[ 70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129]


In [192]:
# investigate transformation between this trajectories space and the robot space
# it is actually a translation, since the orientation in the experiment and the robot space are the same

exp_pt_a = np.array(segment_1[0])
exp_pt_b = np.array(segment_2[-1])
y_dist = exp_pt_a[1]-exp_pt_b[1]
pt_translate = np.array([0,y_dist,0])
exp_pt_b_new = exp_pt_a-pt_translate
# print(y_dist)

exp_seg = np.array([exp_pt_a,exp_pt_b_new])
# print('experiment space segment: ',exp_seg)

right = np.array([0.4, 0.4, 0.5])*1000
left = np.array([-0.4, 0.4, 0.4])*1000
left_new = right-pt_translate

translation = exp_pt_a - right

rob_seg = np.array([right, left_new])
# print('robot space segment: ',rob_seg)

print("Experiment origin :", exp_pt_a)
print("Translation: ", translation)


new_exp_pt_a = exp_pt_a - translation

print("New exp point a: ", new_exp_pt_a)

new_exp_seg_1 = np.array(segment_1)-translation
new_exp_seg_2 = np.array(segment_2)-translation

plot_3d([segment_1, new_exp_seg_1, segment_2, new_exp_seg_2],True)

# # for visualization
# translated_exp_pt_b = exp_pt_b-translation
# translated_exp_pt_a = exp_pt_a-translation
# translated_exp_seg = np.array([translated_exp_pt_a, translated_exp_pt_b])

# plot_3d([translated_exp_seg, rob_seg],True)

('Experiment origin :', array([-911.30200195,  527.66555786,  798.14041138]))
('Translation: ', array([-1311.30200195,   127.66555786,   298.14041138]))
('New exp point a: ', array([ 400.,  400.,  500.]))


In [167]:
# linear regression for multiple axes
# also visualize result in comparison with real data
reg = LinearRegression(n_jobs=-1)

all_segements = [[segment_1, t_1], [segment_2, t_2]]
new_seg = []
all_curves = []
# all_curves = [full_path] # to show the full path in the 3D plot
all_coeffs = []
for segment in all_segements:
    seg, t = segment
    feats = make_features(t, 5) # we are making features from the time steps
    axis_coeffs = []
    for axis in range(3):
        ax_vals = seg[:,axis]
        reg = LinearRegression()
        reg = reg.fit(feats, ax_vals)
        new_ax_vals = reg.predict(feats)
        new_seg.append(new_ax_vals)
        axis_coeffs.append(reg.coef_)
    
        
    all_curves.append(np.transpose(new_seg))
    all_coeffs.append(axis_coeffs)
    new_seg = []
    

ax = plot_3d(all_curves, verbose=True)

In [159]:
for coeffs_ in all_coeffs:
    print(coeffs_)

[array([  0.00000000e+00,   7.15555716e-01,   9.80121204e-02,
        -2.99304190e-03,   2.55294607e-05,  -5.81272425e-08]), array([  0.00000000e+00,  -5.00247803e-01,  -1.04861010e-01,
        -6.04470274e-04,   4.20505344e-05,  -2.98572636e-07]), array([  0.00000000e+00,   4.74966300e-01,   9.33753464e-02,
         1.98458998e-04,  -4.82505137e-05,   4.21923235e-07])]
[array([  0.00000000e+00,  -1.62618121e+02,   3.13652809e+00,
        -3.00581794e-02,   1.42017800e-04,  -2.64411268e-07]), array([  0.00000000e+00,  -1.20413539e+02,   2.15893641e+00,
        -1.87990927e-02,   7.84240567e-05,  -1.24474963e-07]), array([  0.00000000e+00,   6.78664841e+01,  -1.73873675e+00,
         2.10958593e-02,  -1.21240560e-04,   2.66910396e-07])]


In [179]:
# 1 time step = 0.01s
time_step = 0.01
seg_1_coeffs = all_coeffs[0]
seg_2_coeffs = all_coeffs[1]

def seg_params(coeffs, T, p_0=None):
    if not p_0:
        p_0 = coeffs[0]
    v_0 = coeffs[1]
    a_0 = 2*coeffs[2]
    p_T = p_0 + coeffs[1]*T + coeffs[2]*T**2 + coeffs[3]*T**3 + coeffs[4]*T**4 + coeffs[5]*T**5
    v_T = v_0 + 2*coeffs[2]*T + 3*coeffs[3]*T**2 + 4*coeffs[4]*T**3 + 5*coeffs[5]*T**4
    a_T = a_0 + 6*coeffs[3]*T + 12*coeffs[4]*T**2 + 20*coeffs[5]*T**3
    return [p_T, v_T, a_T]

T_1 = len(t_1)*time_step

print(seg_params(seg_1_coeffs[0], T_1))

[0.54789444652986985, 0.84840786936608326, 0.18360317921829722]


In [160]:
# what if we split the trajectory into equal segments and try to approximate, 
# will the result be better?

In [161]:
file_id = 0
scene_groups, mid_points = read_c3d(all_files[file_id], 
                                    objs_id_end, 
                                    rh_id_start, 
                                    lh_id_start,
                                    verbose=0)
# save mid_points data
l_mid_point = mid_points[0]
r_mid_point = mid_points[1]
total_steps = len(l_mid_point)

### Visualize arm motion and path of the end point

In [162]:
grp_nos = 3
def update_lines(num, dataLines, lines):
    for i,groups in enumerate(scene_groups[:grp_nos]):
        frame = groups[:,num,:]

        x_data = frame[:,0]
        y_data = frame[:,1]
        z_data = frame[:,2]
        graph_list[i]._offsets3d = (x_data, y_data, z_data)
        
    title.set_text('Wrist Path Trace, time step={}'.format(num))
        
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num])
        line.set_3d_properties(data[2, :num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
# ax = p3.Axes3D(fig)
ax = fig.add_subplot(111, projection='3d')


# the data
data = [np.transpose(r_mid_point)]

# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1], linewidth=2)[0] for dat in data]

# plot initial data for scatter points
colours = ['r', 'b', 'g', 'k']
graph_list = []
for i, groups in enumerate(scene_groups[:grp_nos]):
    x_data = groups[:,0,:][:,0]
    y_data = groups[:,0,:][:,1]
    z_data = groups[:,0,:][:,2]
    graph = ax.scatter(x_data, y_data, z_data, c=colours[i])
    graph_list.append(graph)

title = ax.set_title('Wrist Path Trace')
# Setting the axes properties
ax.set_xlim3d([-1700.0, 0.0])
ax.set_xlabel('X')

ax.set_ylim3d([-600.0, 800.0])
ax.set_ylabel('Y')

ax.set_zlim3d([700.0, 1600.0])
ax.set_zlabel('Z')
ax.azim = 160 #200
ax.elev = 30

# Creating the Animation object
ani = animation.FuncAnimation(fig, update_lines, total_steps, fargs=(data, lines),
                                   interval=0, blit=False)

plt.show()



In [18]:
# save animation:
ani_writer = animation.FFMpegWriter(fps=30, bitrate=1000)
ani.save('data/media_outputs/'+folder_name+'_trace_'+str(file_id)+'.mp4', fps=30)
# ani.save('data/media_outputs/'+folder_name+'_trace.gif', writer='imagemagick', fps=30)

In [ ]:
# display image or animation through here:
# Image(url='data/media_outputs/basic_animation.gif')